In [1]:
from segment_sales_crawling.financial_report_analyzer_soomin import FinancialAnalyzer
from dotenv import load_dotenv
load_dotenv()

True

In [7]:
company_name = "SK텔레콤"

analyzer = FinancialAnalyzer("4925a6e6e69d8f9138f4d9814f56f371b2b2079a")
segment_sales_df = analyzer.analyze_company(company_name)
segment_sales_df

보고서: 분기보고서 (2023.09)
URL: http://dart.fss.or.kr/report/viewer.do?rcpNo=20231114001064&dcmNo=9495461&eleId=13&offset=58163&length=14449&dtd=dart3.xsd

보고서: 사업보고서 (2023.12)
URL: http://dart.fss.or.kr/report/viewer.do?rcpNo=20240318000570&dcmNo=9725722&eleId=13&offset=205191&length=12854&dtd=dart4.xsd

보고서: 분기보고서 (2024.03)
URL: http://dart.fss.or.kr/report/viewer.do?rcpNo=20240516000420&dcmNo=9947241&eleId=13&offset=60694&length=14388&dtd=dart4.xsd

보고서: 반기보고서 (2024.06)
URL: http://dart.fss.or.kr/report/viewer.do?rcpNo=20240814002241&dcmNo=10067583&eleId=13&offset=204722&length=13104&dtd=dart4.xsd

보고서: 분기보고서 (2024.09)
URL: http://dart.fss.or.kr/report/viewer.do?rcpNo=20241114002758&dcmNo=10186462&eleId=13&offset=55873&length=13099&dtd=dart4.xsd

경고: 7개의 추가 값이 무시되었습니다.
경고: 7개의 추가 값이 무시되었습니다.


,계정,분기보고서 2023.09,사업보고서 2023.12,분기보고서 2024.03,반기보고서 2024.06,분기보고서 2024.09,단위
0,사업부문,9986679,6614124,3338838,13123166,9733333,백만원
1,무선통신 사업,13123166,2018087,1003694,3928020,12942316,백만원
2,유선통신 사업,12942316,264759,132079,557325,12718473,백만원
3,기타사업,3041080,8896970,4474611,17608511,2930678,백만원
4,합 계,3928020,0,0,0,3812989,백만원


In [3]:
# 숫자형 열만 선택하여 10**8 곱하기
numeric_columns = segment_sales_df.select_dtypes(include=['float64', 'int64']).columns
segment_sales_df[numeric_columns] = segment_sales_df[numeric_columns] * (10**8)

In [ ]:
segment_sales_df.drop(columns=['단위'], inplace=True)

In [5]:
import OpenDartReader

dart = OpenDartReader("4925a6e6e69d8f9138f4d9814f56f371b2b2079a")

In [ ]:
sorted_df = dart.list('삼성전자', start='2023-01-01', kind='A').sort_values('rcept_dt', ascending=True)

In [7]:
from segment_sales_crawling.financial_report_analyzer_soomin import ReportFilter

filtered_df = ReportFilter.filter_annual_reports(sorted_df)

In [8]:
import re

oldest_report = filtered_df.iloc[0]['report_nm']
oldest_year = int(re.search(r'\((\d{4})', oldest_report).group(1))
oldest_month = int(re.search(r'\.(\d{2})', oldest_report).group(1))

In [9]:
latest_report = filtered_df.iloc[-1]['report_nm']
latest_year = int(re.search(r'\((\d{4})', latest_report).group(1))
latest_month = int(re.search(r'\.(\d{2})', latest_report).group(1))

In [10]:
report_code_dict = {
    3: [11013,11012,11014,11011,11013], # 3월일 경우 : 1분기, 반기, 3분기, 사업, 1분기
    6: [11012,11014,11011,11013,11012], # 6월일 경우 : 반기, 3분기, 사업, 1분기, 반기
    9: [11014,11011,11013,11012,11014], # 9월일 경우 : 3분기, 사업, 1분기, 반기, 3분기
    12: [11011,11013,11012,11014,11011], # 12월일 경우 : 사업, 1분기, 반기, 3분기, 사업
    }

In [11]:
def str_to_int(value):
    if isinstance(value, str):
        return int(value.replace(',', ''))
    return value

In [12]:
company_name = "NAVER"

n = 5-latest_month//3 
# 9월 = 3분기 -> 5-3 = 2 -> 3분기,4분기 = 이전 년도 2개 분기값 필요
# 6월 = 2분기 -> 5-2 = 3 -> 2분기,3분기,4분기 = 이전 년도 3개 분기값 필요
# 3월 = 1분기 -> 5-1 = 4 -> 1분기,2분기,3분기,4분기 = 이전 년도 4개 분기값 필요
# 12월 = 4분기 -> 5-4 = 1 -> 4분기 = 이전 년도 1개 분기값 필요

save_values_dict = {}

for i in report_code_dict[latest_month][:n]:
    df = dart.finstate(company_name,oldest_year,reprt_code=str(i))
    if str(i) == '11011':
        sales = str_to_int(df.loc[(df['account_nm'] == '매출액') & (df['fs_nm'] == '연결재무제표')]['thstrm_amount'].iloc[0])               
        operating_profit = str_to_int(df.loc[(df['account_nm'] == '영업이익') & (df['fs_nm'] == '연결재무제표')]['thstrm_amount'].iloc[0])
        net_profit = str_to_int(df.loc[(df['account_nm'] == '당기순이익') & (df['fs_nm'] == '연결재무제표')]['thstrm_amount'].iloc[0])
        save_values_dict[str(oldest_year)+"-"+str(i)] = [sales, operating_profit, net_profit]
    else:
        sales = str_to_int(df.loc[(df['account_nm'] == '매출액') & (df['fs_nm'] == '연결재무제표')]['thstrm_add_amount'].iloc[0])
        operating_profit = str_to_int(df.loc[(df['account_nm'] == '영업이익') & (df['fs_nm'] == '연결재무제표')]['thstrm_add_amount'].iloc[0])
        net_profit = str_to_int(df.loc[(df['account_nm'] == '당기순이익') & (df['fs_nm'] == '연결재무제표')]['thstrm_add_amount'].iloc[0])
        save_values_dict[str(oldest_year)+"-"+str(i)] = [sales, operating_profit, net_profit]

In [13]:
for i in report_code_dict[latest_month][n:5]:
    df = dart.finstate(company_name,latest_year,reprt_code=str(i))
    if str(i) == '11011':
        sales = str_to_int(df.loc[(df['account_nm'] == '매출액') & (df['fs_nm'] == '연결재무제표')]['thstrm_amount'].iloc[0])
        operating_profit = str_to_int(df.loc[(df['account_nm'] == '영업이익') & (df['fs_nm'] == '연결재무제표')]['thstrm_amount'].iloc[0])
        net_profit = str_to_int(df.loc[(df['account_nm'] == '당기순이익') & (df['fs_nm'] == '연결재무제표')]['thstrm_amount'].iloc[0])
        save_values_dict[str(latest_year)+"-"+str(i)] = [sales, operating_profit, net_profit]
    else:
        sales = str_to_int(df.loc[(df['account_nm'] == '매출액') & (df['fs_nm'] == '연결재무제표')]['thstrm_add_amount'].iloc[0])
        operating_profit = str_to_int(df.loc[(df['account_nm'] == '영업이익') & (df['fs_nm'] == '연결재무제표')]['thstrm_add_amount'].iloc[0])
        net_profit = str_to_int(df.loc[(df['account_nm'] == '당기순이익') & (df['fs_nm'] == '연결재무제표')]['thstrm_add_amount'].iloc[0])
        save_values_dict[str(latest_year)+"-"+str(i)] = [sales, operating_profit, net_profit]


In [ ]:
save_values_dict

In [ ]:
import pandas as pd

df = pd.DataFrame(save_values_dict).T.transpose()
new_column_name = "사업부문"
df.insert(0, new_column_name, ['영업수익', '영업이익', '순이익'])
df


In [ ]:
df.columns = result_df.columns
final_df = pd.concat([result_df, df], axis=0, ignore_index=True)
final_df
